# functions: write_results

[write_results](../api/functions.rst#nornir_utils.plugins.functions.write_results) is a function that writes result to files with hostname names and returns list of tuples with hostname + diff between old file and new file:

In [1]:
from nornir import InitNornir

def dummy_task(task):
    return "hi!!!"

nr = InitNornir(
    inventory={
        "plugin":"YAMLInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        }
    }
)

result = nr.run(task=dummy_task)

Now we could write the results easily with the `write_results` function:

In [2]:
from nornir_utils.plugins.functions import write_results

write_results(result, dirname="out_files/write_results")

[('dev1.group_1',
  '--- out_files/write_results/dev1.group_1\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev1.group_1: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!'),
 ('dev2.group_1',
  '--- out_files/write_results/dev2.group_1\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev2.group_1: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!'),
 ('dev3.group_2',
  '--- out_files/write_results/dev3.group_2\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev3.group_2: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!'),
 ('dev4.group_2',
  '--- out_files/write_results/dev4.group_2\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev4.group_2: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!'),
 ('dev5.no_group',
  '--- out_files/write_results/dev5.no_group\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev5.no_group: dummy_task ** changed : False ------------------------------ INFO\n+\n+hi!!!')]

```text
$ ls out_files/write_results

dev1.group_1  dev2.group_1  dev3.group_2  dev4.group_2	dev5.no_group
```

Let's look at the content of each file:

In [3]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dev1.group_1: dummy_task ** changed : False ------------------------------- INFO

hi!!!
out_files/write_results/dev2.group_1
---- dev2.group_1: dummy_task ** changed : False ------------------------------- INFO

hi!!!
out_files/write_results/dev3.group_2
---- dev3.group_2: dummy_task ** changed : False ------------------------------- INFO

hi!!!
out_files/write_results/dev4.group_2
---- dev4.group_2: dummy_task ** changed : False ------------------------------- INFO

hi!!!
out_files/write_results/dev5.no_group
---- dev5.no_group: dummy_task ** changed : False ------------------------------ INFO

hi!!!


`dirname` here is path to directory, you want to write the results for each host. `write_results` creates directories from the path, if it's necessary.

## Diff

As you can see, `write_results` returns list of tuples with hostname + diff between the previous and current file state:

In [4]:
def smart_task(task):
    return "hihi!!!"

result = nr.run(task=smart_task)

print(write_results(result, dirname="out_files/write_results"))

[('dev1.group_1', '--- out_files/write_results/dev1.group_1\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev1.group_1: smart_task ** changed : False ------------------------------- INFO\n+\n+hihi!!!'), ('dev2.group_1', '--- out_files/write_results/dev2.group_1\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev2.group_1: smart_task ** changed : False ------------------------------- INFO\n+\n+hihi!!!'), ('dev3.group_2', '--- out_files/write_results/dev3.group_2\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev3.group_2: smart_task ** changed : False ------------------------------- INFO\n+\n+hihi!!!'), ('dev4.group_2', '--- out_files/write_results/dev4.group_2\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev4.group_2: smart_task ** changed : False ------------------------------- INFO\n+\n+hihi!!!'), ('dev5.no_group', '--- out_files/write_results/dev5.no_group\n\n+++ new\n\n@@ -0,0 +1,3 @@\n\n+---- dev5.no_group: smart_task ** changed : False ------------------------------ INFO\n+\n+hihi!!!')]


In [5]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dev1.group_1: smart_task ** changed : False ------------------------------- INFO

hihi!!!
out_files/write_results/dev2.group_1
---- dev2.group_1: smart_task ** changed : False ------------------------------- INFO

hihi!!!
out_files/write_results/dev3.group_2
---- dev3.group_2: smart_task ** changed : False ------------------------------- INFO

hihi!!!
out_files/write_results/dev4.group_2
---- dev4.group_2: smart_task ** changed : False ------------------------------- INFO

hihi!!!
out_files/write_results/dev5.no_group
---- dev5.no_group: smart_task ** changed : False ------------------------------ INFO

hihi!!!


## Writing specific data

If the task returns different information, you can also select which ones to write. For instance:

In [6]:
from nornir.core.task import Result

def task_with_data(task):
    return Result(host=task.host, result="Spaghetti monster", sw_char="Jar Jar Binks", food="hawaiian pizza", OS="windows")


fav_result = nr.run(task=task_with_data)

# write only sw_char, food vars and write diff to diff variable 
diffs = write_results(fav_result, dirname="out_files/write_results", vars=["sw_char", "food"])

In [7]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dev1.group_1: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza
out_files/write_results/dev2.group_1
---- dev2.group_1: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza
out_files/write_results/dev3.group_2
---- dev3.group_2: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza
out_files/write_results/dev4.group_2
---- dev4.group_2: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza
out_files/write_results/dev5.no_group
---- dev5.no_group: task_with_data ** changed : False -------------------------- INFO

Jar Jar Binks

hawaiian pizza


## Severity

You can also ask `write_results` to write the results only if the severity is equal or superior to the one specified. This is particularly useful if your script is very large and you don't care about intermediate results. For instance:

In [8]:
import logging

def complex_task(task):
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    return "I did a lot of things!!!"

complex_result = nr.run(task=complex_task)
complex_result

AggregatedResult (complex_task): {'dev1.group_1': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev2.group_1': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev3.group_2': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev4.group_2': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev5.no_group': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"]}

`write_results` will only write `INFO` severity and above by default so if we write the result we should only see the result of the parent task:

In [9]:
diffs = write_results(complex_result, dirname="out_files/write_results")

In [10]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dev1.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!
out_files/write_results/dev2.group_1
---- dev2.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!
out_files/write_results/dev3.group_2
---- dev3.group_2: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!
out_files/write_results/dev4.group_2
---- dev4.group_2: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!
out_files/write_results/dev5.no_group
---- dev5.no_group: complex_task ** changed : False ---------------------------- INFO

I did a lot of things!!!


If you wanted to write all the tasks for debugging purposes you could specify it with:

In [11]:
diffs = write_results(complex_result, dirname="out_files/write_results", severity_level=logging.DEBUG)

In [12]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dev1.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!
out_files/write_results/dev2.group_1
---- dev2.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev2.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev2.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev2.group_1: dummy_task ** cha

## Write host

`write_host` argument can help you, if you don't need to write hostnames to files:

In [13]:
diff = write_results(result, dirname="out_files/write_results", write_host=False)

In [14]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!
out_files/write_results/dev2.group_1
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!
out_files/write_results/dev3.group_2
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!
out_files/write_results/dev4.group_2
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!
out_files/write_results/dev5.no_group
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!


## Append

`append` argument allows you to use `a+` or `w+` (default mode) writing mode. As instance:

In [15]:
diffs = write_results(fav_result, dirname="out_files/write_results", append=True)

In [16]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- dev1.group_1: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster
out_files/write_results/dev2.group_1
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- dev2.group_1: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster
out_files/write_results/dev3.group_2
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- dev3.group_2: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster
out_files/write_results/dev4.group_2
---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- dev4.group_2: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster
out_files/write_results/dev5.no_group
---- smart

# No errors

Don't write errors to files with `no_errors` argument. As instance:

In [17]:
def task_with_exception(task):
    return Result(host=task.host, result="Something went wrong", exception=Exception())


def task_without_exception(task):
    return "All is fine"

def task(task):
    task.run(task=task_with_exception)
    task.run(task=task_without_exception)
    return "What's happening?"

    
result_with_exception = nr.run(task=task)

diffs = write_results(result_with_exception, dirname="out_files/write_results")

In [18]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dev1.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev1.group_1: task_with_exception ** changed : False ---------------------- INFO

Something went wrong

---- dev1.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine
out_files/write_results/dev2.group_1
---- dev2.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev2.group_1: task_with_exception ** changed : False ---------------------- INFO

Something went wrong

---- dev2.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine
out_files/write_results/dev3.group_2
---- dev3.group_2: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev3.group_2: task_with_exception ** changed : False ---------------------- INFO

Something went wrong

---- dev3.group_2: task_without_exception ** c

In [19]:
diffs = write_results(result_with_exception, dirname="out_files/write_results", no_errors=True)

In [20]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dev1.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev1.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine
out_files/write_results/dev2.group_1
---- dev2.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev2.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine
out_files/write_results/dev3.group_2
---- dev3.group_2: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev3.group_2: task_without_exception ** changed : False ------------------- INFO

All is fine
out_files/write_results/dev4.group_2
---- dev4.group_2: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev4.group_2: task_without_exception ** changed : False ------------------- INFO

All is fine
out_files/write_results/dev5.no_